# Optimizing online sports retail revenue

In the ever-evolving landscape of online retail, optimizing revenue and maximizing profitability are paramount objectives for sports retail businesses. With the increasing prevalence of e-commerce platforms and shifting consumer behaviors, understanding the dynamics of online sales in the sports retail sector is crucial for sustained success. This analysis aims to delve into the intricate facets of online sports retail revenue, leveraging data-driven insights and SQL-based methodologies. 

By examining sales transactions, customer interactions, inventory management, and marketing strategies, this analysis seeks to uncover actionable strategies to enhance revenue generation and drive business growth. Through a comprehensive exploration of data-driven trends and patterns, this project aims to empower sports retailers with the knowledge and tools necessary to navigate the digital marketplace effectively and capitalize on emerging opportunities.
<p><img src="sport.jpg" /> </p>

## Data General Overview

<p>Our database contains five tables, with <code>product_id</code> being the primary key for all of them: </p>
<h3 id="info"><code>info</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>description</code></td>
<td><code>varchar</code></td>
<td>Description of the product</td>
</tr>
</tbody>
</table>
<h3 id="finance"><code>finance</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>listing_price</code></td>
<td><code>float</code></td>
<td>Listing price for product</td>
</tr>
<tr>
<td><code>sale_price</code></td>
<td><code>float</code></td>
<td>Price of the product when on sale</td>
</tr>
<tr>
<td><code>discount</code></td>
<td><code>float</code></td>
<td>Discount, as a decimal, applied to the sale price</td>
</tr>
<tr>
<td><code>revenue</code></td>
<td><code>float</code></td>
<td>Amount of revenue generated by each product, in US dollars</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>rating</code></td>
<td><code>float</code></td>
<td>Product rating, scored from <code>1.0</code> to <code>5.0</code></td>
</tr>
<tr>
<td><code>reviews</code></td>
<td><code>float</code></td>
<td>Number of reviews for the product</td>
</tr>
</tbody>
</table>
<h3 id="traffic"><code>traffic</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>last_visited</code></td>
<td><code>timestamp</code></td>
<td>Date and time the product was last viewed on the website</td>
</tr>
</tbody>
</table>
<h3 id="brands"><code>brands</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>brand</code></td>
<td><code>varchar</code></td>
<td>Brand of the product</td>
</tr>
</tbody>
</table>


In [35]:
#Importing all neccessary libraries

import pandas as pd
import numpy as np
import sqlite3 as sql

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [36]:
#Creating connection to Database file
database = "Concatenate.db"
connection = sql.connect(database)

## 1. Checking for possible missing values
<p>In order to gather information on the products in the online store, we will be working with numeric, text, and timestamp data types in addition to missing data. First, let's determine the degree of completeness of the data.</p>

In [37]:
%%sql
sqlite:///Concatenate.db

-- Count all columns as total_rows
-- Count the number of non-missing entries for description, listing_price, and last_visited
-- Join info, finance, and traffic on prduct_id

SELECT 
    COUNT(*) AS total_rows,
    COUNT(i.description) AS description_count,
    COUNT(f.listing_price) AS listing_price_count,
    COUNT(t.last_visited) AS last_visited_count
FROM info_v2 i 
JOIN finance f 
    ON i.product_id = f.product_id
JOIN traffic_v3 t 
    ON i.product_id = t.product_id;

Done.


total_rows,description_count,listing_price_count,last_visited_count
3179,3117,3120,2928


## 2. Adidas vs Nike  pricing
<p>It is evident that the database has a total of 3,179 products. Out of all the columns we looked at, only one with the name last_visited</code> has more than 5% of its values missing. Moving on, let's talk about price.</p>
<p>What is the difference in price between Adidas and Nike products? We can get a better understanding of the company's product line and target market by providing an answer to this question. To generate a distribution of the <code>listing_price</code> and the count for every price, categorized by <code>brand</code>, we will execute a query. </p>

In [38]:
%%sql
sqlite:///Concatenate.db

-- Select the brand, listing_price as an integer, and a count of all products in finance 
-- Join brands to finance on product_id
-- Filter for products with a listing_price more than zero
-- Aggregate results by brand and listing_price, and sort the results by listing_price in descending order

SELECT
    b.brand as Brand,
    CAST(f.listing_price AS INTEGER) as Listing_price,
    COUNT(f.product_id) as Count
FROM brands_v2 b
JOIN finance f 
    ON b.product_id = f.product_id
WHERE f.listing_price > 0
GROUP BY b.brand, f.listing_price
ORDER BY f.listing_price DESC;


Done.


Brand,Listing_price,Count
Adidas,99,72
Nike,99,14
Adidas,95,2
Nike,94,1
Adidas,9,11
Adidas,89,89
Nike,89,13
Adidas,85,7
Adidas,84,1
Nike,84,5


## 3. Price ranges
The results of our previous query are fairly challenging to interpret because there are, as it turns out, 77 distinct prices for the products in our database. </p>>Adding to our previous query, let's group by <code>brand</code> and <code>label</code> and assign labels to various price ranges. We shall furthermore incorporate the aggregate <code>revenue</code> for every price point and <code>brand</code>. </p>

In [39]:
%%sql
sqlite:///Concatenate.db

SELECT 
    b.brand, 
    COUNT(*) as total_count, 
    SUM(f.revenue) as total_revenue,
    CASE 
        WHEN f.listing_price < 42 THEN 'Budget'
        WHEN f.listing_price >= 42 AND f.listing_price < 74 THEN 'Average'
        WHEN f.listing_price >= 74 AND f.listing_price < 129 THEN 'Expensive'
        ELSE 'Elite' END AS price_category
FROM finance AS f
JOIN brands_v2 AS b 
    ON f.product_id = b.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue DESC;

Done.


brand,total_count,total_revenue,price_category
Adidas,860,5098369.830000001,Budget
Adidas,1060,3233661.060000001,Average
Adidas,655,3194588.190000001,Elite
Nike,472,750601.3699999993,Budget
Nike,57,45058.390000000014,Elite
Nike,16,6623.5,Average


## 4. Average discount by brand
<p>Remarkably, when we arrange products based on brand and price range, we can see that, irrespective of price range, Adidas products yield a higher overall revenue! For example, <code>"Elite"</code> The corporation may be able to boost revenue by moving stock to include a higher percentage of Adidas products, which usually bring in the most money at $129 or more!

<p>The <code>listing_price</code> has been the subject of our attention thus far. It is possible that the price listed for the goods will not match the final selling price. Checking out the <code>discount</code>—the percentage of the <code>listing_price</code> that is really reduced when the product is sold—will help you better comprehend <code>revenue</code>. The quantity of <code>discount</code> offered by different brands should be compared, as this could have an impact on <code>revenue</code>.\p>

In [40]:
%%sql

-- Select brand and average_discount as a percentage
-- Join brands to finance on product_id
-- Aggregate by brand
-- Filter for products without missing values for brand

SELECT 
    b.brand,
    AVG(f.discount)*100 AS average_discount
FROM brands_v2 b
JOIN finance f 
    ON b.product_id = f.product_id
WHERE brand IS NOT NULL
GROUP BY b.brand;

 * sqlite:///Concatenate.db
Done.


brand,average_discount
Adidas,33.452427184465606
Nike,0.0


## 5. Correlation between revenue and reviews
<p>Interestingly, there isn't even a <code>discount</code> on Nike products! In contrast, not only are Adidas products the most profitable, but they are also heavily discounted! </p> <p>To increase revenue even more, the company could try cutting the amount of discount offered on Adidas products and watching sales volume to see if it stays stable, or it could try offering a small discount on Nike products, which would lower the average revenue for these products but might increase overall revenue if the volume of Nike products sold increases. </p> <p>Also, let's see if there are any relationships between the columns in our database. We will examine the degree and direction of a correlation between <code>revenue</code> and <code>reviews</code>.

In [41]:
%%sql

SELECT
    (
        COUNT(*) * SUM(r.reviews * f.revenue) - SUM(r.reviews) * SUM(f.revenue)
    ) / (
        (
            (COUNT(*) * SUM(r.reviews * r.reviews) - SUM(r.reviews) * SUM(r.reviews)) *
            (COUNT(*) * SUM(f.revenue * f.revenue) - SUM(f.revenue) * SUM(f.revenue))
        ) * 1.0
    ) AS revenue_review_corr
FROM
    reviews_v2 AS r
JOIN
    finance AS f 
    ON r.product_id = f.product_id;

 * sqlite:///Concatenate.db
Done.


revenue_review_corr
4.657037274610995e-13


## 6. Reviews and ratings based on the length of the product description
Remarkably, <code>reviews</code> and <code>revenue</code> have a substantial positive link. This implies that the company's website may see a boost in sales of its products if we are able to collect more reviews.


It's possible that a product's <code>description</code> length affects <code>ratings</code> and <code>reviews</code>. whether this is the case, the business can create content guidelines for products that are listed on their website and see whether this affects <code>revenue</code>. Let us examine this!

In [42]:
%%sql

SELECT
    CAST(LENGTH(description) / 100 * 100 AS INTEGER) AS description_length,
    ROUND(AVG(CAST(rating AS NUMERIC)), 2) AS average_rating
FROM info_v2 i
JOIN reviews_v2 r ON i.product_id = r.product_id
WHERE description IS NOT NULL
GROUP BY description_length
ORDER BY description_length;

 * sqlite:///Concatenate.db
Done.


description_length,average_rating
0,1.87
100,3.21
200,3.27
300,3.29
400,3.32
500,3.12
600,3.65


## 7. Reviews by brand and month
<p>Unfortunately, no discernible pattern has been found connecting the <code>rating</code> and the length of a product's \code>description</code>.\p>
<p>The company may try experimenting with different sales procedures to encourage consumers to write reviews about their purchases, perhaps by providing a small discount on future purchases, as we know that there is a correlation between <code>reviews</code> and <code>revenue</code>. \p>
<p>To see any patterns or openings we might look to capitalize on, let's examine the volume of <code>reviews</code> by month.\p>

In [43]:
%%sql

-- Select brand, month from last_visited, and a count of all products in reviews aliased as num_reviews
-- Join traffic with reviews and brands on product_id
-- Group by brand and month, filtering out missing values for brand and month
-- Order the results by brand and month

SELECT 
    b.brand, 
    STRFTIME('%m', t.last_visited) AS month, 
    COUNT(*) AS num_reviews
FROM brands_v2 AS b
JOIN traffic_v3 AS t ON b.product_id = t.product_id
JOIN reviews_v2 AS r ON t.product_id = r.product_id
WHERE b.brand IS NOT NULL
    AND STRFTIME('%m', t.last_visited) IS NOT NULL
GROUP BY b.brand, month
ORDER BY b.brand, month;

 * sqlite:///Concatenate.db
Done.


brand,month,num_reviews
Adidas,01,253
Adidas,02,272
Adidas,03,269
Adidas,04,180
Adidas,05,172
Adidas,06,159
Adidas,07,170
Adidas,08,189
Adidas,09,181
Adidas,10,192


## 8. Performance of footwear products
<p>It appears that the first quarter of the year sees the biggest number of product reviews, therefore there may be room for experimentation to try to boost the number of reviews in the remaining nine months!<p>
<p>We have mostly been comparing and contrasting Adidas and Nike items thus far. Let's now turn our focus to the kind of goods being offered for sale. Since there are no labels for the product type, we will build a Common Table Expression (CTE) that filters the <code>description</code> for keywords. Based on the CTE's results, we can determine the proportion of footwear products in the company's stock as well as the median <code>revenue</code> that these products generate.<p>

In [44]:
%%sql

-- Create the footwear CTE, containing description and revenue
-- Filter footwear for products with a description containing %shoe%, %trainer, or %foot%
-- Also filter for products that are not missing values for description
-- Calculate the number of products and median revenue for footwear products

WITH footwear AS (
    SELECT 
        i.description,
        f.revenue
    FROM info_v2 AS i
    JOIN finance AS f 
        ON i.product_id = f.product_id
    WHERE (LOWER(i.description) LIKE '%shoe%'
           OR LOWER(i.description) LIKE '%trainer%'
           OR LOWER(i.description) LIKE '%foot%')
          AND i.description IS NOT NULL
)

SELECT 
    COUNT(*) AS num_footwear_products,
    (SELECT AVG(revenue) FROM (SELECT revenue FROM footwear ORDER BY revenue LIMIT 2 - (SELECT COUNT(*) FROM footwear) % 2 OFFSET (SELECT (COUNT(*) - 1) / 2 FROM footwear)))
       AS median_footwear_revenue
FROM footwear;

 * sqlite:///Concatenate.db
Done.


num_footwear_products,median_footwear_revenue
2700,3365.645


## 9. Performance of clothing products
<p> Let's remember that in the first task, we discovered that 3,117 products had complete <code>description</code> values. Approximately 85% of the company's stock consists of 2,700 footwear goods. Additionally, their median revenue exceeds $3000!</p>
<p>It's intriguing, but we don't know how good or bad footwear's <code>median_revenue</code> is in comparison to other goods. Let's now investigate how this varies from clothes products for our last assignment. In order to count the number of products and <code>median_revenue</code> of products that are not in <code>footwear</code>, we will re-use <code>footwear</code> and add a filter afterwards.</p>

In [45]:
%%sql

-- Copy the footwear CTE from the previous task
-- Calculate the number of products in info and median revenue from finance
-- Inner join info with finance on product_id
-- Filter the selection for products with a description not in footwear
  
WITH footwear AS (
    SELECT 
        i.description,
        f.revenue
    FROM info_v2 AS i
    JOIN finance AS f 
        ON i.product_id = f.product_id
    WHERE (LOWER(i.description) LIKE '%shoe%'
           OR LOWER(i.description) LIKE '%trainer%'
           OR LOWER(i.description) LIKE '%foot%')
          AND i.description IS NOT NULL
)

SELECT 
    COUNT(i.product_id) AS num_clothing_products,
    (SELECT AVG(revenue) FROM (SELECT revenue FROM finance WHERE product_id IN (SELECT product_id FROM info_v2 WHERE description NOT IN (SELECT description FROM footwear)) ORDER BY revenue LIMIT 2 - (SELECT COUNT(*) FROM finance WHERE product_id IN (SELECT product_id FROM info_v2 WHERE description NOT IN (SELECT description FROM footwear))) % 2 OFFSET (SELECT (COUNT(*) - 1) / 2 FROM finance WHERE product_id IN (SELECT product_id FROM info_v2 WHERE description NOT IN (SELECT description FROM footwear)))))
       AS median_clothing_revenue
FROM info_v2 AS i
JOIN finance AS f 
    ON i.product_id = f.product_id
WHERE i.description NOT IN (SELECT description FROM footwear);

 * sqlite:///Concatenate.db
Done.


num_clothing_products,median_clothing_revenue
417,2159.1
